# Basic EM workflow 3 (Restaurants data set)

# Introduction

This IPython notebook explains a basic workflow two tables using *py_entitymatching*. Our goal is to come up with a workflow to match restaurants from Fodors and Zagat sites. Specifically, we want to maximize F1. The datasets contain information about the restaurants.

First, we need to import *py_entitymatching* package and other libraries as follows:

In [1]:
import sys
#sys.path.append('/Users/pradap/Documents/Research/Python-Package/anhaid/py_entitymatching/')

import py_entitymatching as em
import pandas as pd
import os

In [2]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.6.5 (default, Apr  1 2018, 05:46:30) 
[GCC 7.3.0]
pandas version: 0.22.0
magellan version: 0.3.0


Matching two tables typically consists of the following three steps:

** 1. Reading the input tables **

** 2. Blocking the input tables to get a candidate set **

** 3. Matching the tuple pairs in the candidate set **

# Read input tables

We begin by loading the input tables. For the purpose of this guide, we use the datasets that are included with the package.

In [3]:
# Get the paths
path_A = '/home/liang/Workspace/cs839-project/stage-2/crawlers/data/amazon_products.csv'
path_B = '/home/liang/Workspace/cs839-project/stage-2/crawlers/data/walmart_products.csv'

In [4]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [5]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 2976
Number of tuples in B: 4847
Number of tuples in A X B (i.e the cartesian product): 14424672


In [6]:
A.head(2)

,id,product title,average battery life,brand,hard disk size,model,operating system,processor (cpu) manufacturer,processor speed,ram size,screen size,wireless type,combo
0,a0,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,7 hours,acer,256.0,nh.q28aa.001,windows 10,3.8 ghz intel core i7,3.8 ghz,16 gb,15.6in,802.11 a/c,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...
1,a1,acer aspire e 15 15.6 full hd 8th gen intel core i5-8250u geforce mx150 8gb ram memory 256gb ssd...,15 hours,acer,256.0,e5-576g-5762,windows 10,1.6 ghz intel core i5,1.6 ghz,8 gb,15.6in,NaN,acer aspire e 15 15.6 full hd 8th gen intel core i5-8250u geforce mx150 8gb ram memory 256gb ssd...


In [7]:
B.head(2)

,id,product title,average battery life,brand,hard disk size,model,operating system,processor (cpu) manufacturer,processor speed,ram size,screen size,wireless type,combo
0,w0,iview i896qw 8.95 2-in-1 32gb tablet intel atom bay trail z3735f processor windows 10,NaN,iview,NaN,NaN,microsoft windows,NaN,NaN,NaN,8.95,NaN,iview i896qw 8.95 2-in-1 32gb tablet intel atom bay trail z3735f processor windows 10 iview micr...
1,w1,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red,NaN,dell,500 kb,i3168-3270red,windows 10,intel pentium,1.6 hz,4 kb,11.6,NaN,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...


In [8]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('id', 'id')

# Block tables to get candidate set

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.
*py_entitymatching* provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. The user can mix and match these blockers to form a blocking sequence applied to input tables.

For the matching problem at hand, we know that two restaurants with different city names will not match. So we decide the apply blocking over names:

In [9]:
# Blocking plan

# A, B -- attribute equiv. blocker [city] --------------------|---> candidate set

# block on: brand, ram, OS?, model? 

In [10]:
ab = em.AttrEquivalenceBlocker()
#For the given two tables, we will assume that two persons with different zipcode values do not refer to the same real world person. So, we apply attribute equivalence blocking on zipcode. That is, we block all the tuple pairs that have different zipcodes.

# Use block_tables to apply blocking over two input tables.
C1 = ab.block_tables(A, B, 
                    l_block_attr='brand', r_block_attr='brand', 
                    l_output_attrs=['id','brand', 'combo'],
                    r_output_attrs=['id','brand', 'combo'],
                    n_jobs=-1)
len(C1)

1636110

In [11]:
C1.head()

,_id,ltable_id,rtable_id,ltable_brand,ltable_combo,rtable_brand,rtable_combo
0,0,a0,w28,acer,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,acer,manufacturer refurbished acer 11.6 intel celeron 1.1 ghz 4 gb ram 32 gb flash windows 10 home|sp...
1,1,a0,w30,acer,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,acer,switch alpha 12 sa5-271-78m8 12in 8gb 256gb w10h 11ac i7-6500u acer 256 gb nt.lcdaa.014 windows ...
2,2,a0,w48,acer,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,acer,acer switch alpha 12 sa5-271p-5972 - 12 - core i5 6200u - 8 gb ram - 256 gb ssd - us internation...
3,3,a0,w49,acer,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,acer,r5-471t-78vy i7-6500u 2.5g 8gb 256gb ssd 14in 11ac w10h acer 256 gb nx.g7waa.012 windows 10 inte...
4,4,a0,w70,acer,acer predator helios 300 gaming laptop 15.6 full hd intel core i7-7700hq cpu 16gb ddr4 ram 256gb...,acer,acer switch alpha 12 sa5-271p-74e1 - 12 - core i7 6500u - 8 gb ram - 256 gb ssd - us internation...


## Debug blocker output

The number of tuple pairs considered for matching is reduced to 10165 (from 176423), but we would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in *py_entitymatching* as follows:

In [12]:
# # Debug blocker output
# dbg = em.debug_blocker(C1, A, B, output_size=10, 
#                        attr_corres=[
#                            ('product title','product title'),
#                            ('brand', 'brand'), 
#                            ('model','model'),
#                            ('combo','combo')],
#                       verbose=True)
# #### Display first few tuple pairs from the debug_blocker's output
# dbg.head()

From the debug blocker's output we observe that the current blocker drops quite a few potential matches. We would want to update the blocking sequence to avoid dropping these potential matches.

For the considered dataset, we know that for the restaurants to match the  names must overlap between them. We could use overlap blocker for this purpose. Finally, we would want to union the outputs from the attribute equivalence blocker and the overlap blocker to get a consolidated candidate set.

In [13]:
# Updated blocking sequence
# A, B ------ attribute equivalence [city] -----> C1--
#                                                     |----> C
# A, B ------ overlap blocker [name] --------> C2--

In [14]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C2 = ob.block_tables(A,B, 'model', 'model', 
                    l_output_attrs=['id','model','combo'], 
                    r_output_attrs=['id','model','combo'],
                    overlap_size=5,
                    q_val=5,
                    word_level=False,
                    show_progress=True,
                    n_jobs=-1
                    )
len(C2)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


22358

In [15]:
# Display first two rows from C2
C2.head()

,_id,ltable_id,rtable_id,ltable_model,ltable_combo,rtable_model,rtable_combo
0,0,a229,w1,i3168-3271blu,dell i3168-3271blu 11.6 hd 2-in-1 laptop (intel pentium n3710 1.6ghz processor 4 gb ddr3l sdram ...,i3168-3270red,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
1,1,a2596,w1,i3542-5666red,dell inspiron 3000 series 15.6 non-touch notebook (red) intel core i3-4005u 4gb memory 500 gb ha...,i3168-3270red,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
2,2,a229,w4,i3168-3271blu,dell i3168-3271blu 11.6 hd 2-in-1 laptop (intel pentium n3710 1.6ghz processor 4 gb ddr3l sdram ...,i3168-0027red,dell inspiron 11 3168 11.6 laptop touchscreen 2-in-1 windows 10 home intel celeron n3060 process...
3,3,a2596,w4,i3542-5666red,dell inspiron 3000 series 15.6 non-touch notebook (red) intel core i3-4005u 4gb memory 500 gb ha...,i3168-0027red,dell inspiron 11 3168 11.6 laptop touchscreen 2-in-1 windows 10 home intel celeron n3060 process...
4,4,a8,w5,x7s52ua#aba,hp 14 inch stream laptop intel celeron n3060 processor 4gb ram 32gb emmc 1-year office 365 perso...,x7u40ua#aba,hp x2 10-p020nr 10.1 laptop touchscreen 2-in-1 windows 10 home intel atom x5-z8350 processor 2gb...


In [16]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C3 = ob.block_tables(A,B, 'ram size', 'ram size', 
                    l_output_attrs=['id','ram size','combo'], 
                    r_output_attrs=['id','ram size','combo'],
                    overlap_size=5,
                    q_val=5,
                    word_level=False,
                    show_progress=True,
                    n_jobs=-1
                    )
len(C3)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


2169890

In [17]:
C3.head()

,_id,ltable_id,rtable_id,ltable_ram size,ltable_combo,rtable_ram size,rtable_combo
0,0,a1528,w1,4 kb,acer r 11 convertible 2-in-1 11.6 touchscreen chromebook-intel celeron n3150 4gb ddr3l 32gb emmc...,4 kb,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
1,1,a44,w2,2 gb,acer chromebook cb3-131-c3sz 11.6-inch laptop (intel celeron n2840 dual-core processor2 gb ram16...,2 gb,iview maximus ii 11.6 laptop touchscreen 2-in-1 windows 10 intel bay trail z3735f processor 2gb ...
2,2,a118,w2,2 gb,lenovo ideapad 210s 11.6 inch hd flagship laptop (2018 edition)| intel celeron n3350 dual-core u...,2 gb,iview maximus ii 11.6 laptop touchscreen 2-in-1 windows 10 intel bay trail z3735f processor 2gb ...
3,3,a132,w2,2 gb,acer chromebook 11.6-inch cb3-111-c670 (intel celeron 2gb 16gb ssd white) discontinued by manufa...,2 gb,iview maximus ii 11.6 laptop touchscreen 2-in-1 windows 10 intel bay trail z3735f processor 2gb ...
4,4,a177,w2,2 gb,2017 newest premium high performance samsung 11.6 hd chromebook - intel dual-core celeron n3050 ...,2 gb,iview maximus ii 11.6 laptop touchscreen 2-in-1 windows 10 intel bay trail z3735f processor 2gb ...


We add another blocker.

In [18]:
# Create overlap blocker
ob = em.OverlapBlocker()

# Block tables using 'name' attribute 
C4 = ob.block_tables(A, B, 'combo', 'combo', 
                    l_output_attrs=['id','combo'], 
                    r_output_attrs=['id','combo'],
                    overlap_size=50,
                    q_val=5,
                    word_level=False,
                    show_progress=True,
                    n_jobs=-1
                    )
len(C4)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


107883

In [19]:
C4.head()

,_id,ltable_id,rtable_id,ltable_combo,rtable_combo
0,0,a727,w1,2018 flagship dell inspiron 11.6 2-in-1 hd touchscreen convertible laptop/tablet - intel pentium...,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
1,1,a1316,w1,dell flagship inspiron 11.6 hd 2-in-1 convertible touchscreen laptop/tablet- intel quad-core pen...,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
2,2,a1765,w1,dell inspiron i3000-10099slv 11.6 inch touchscreen 2-in-1 laptop (intel pentium 4 gb ram 500 gb ...,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
3,3,a2842,w1,dell - inspiron 2-in-1 15.6 touch-screen laptop (intel core i5 8gb memory 500gb hard drive black...,dell - inspiron 2-in-1 11.6 touch-screen laptop - intel pentium - 4gb memory - 500gb hd - red de...
4,4,a236,w2,lenovo 11.6?? ips touchscreen 2-in-1 convertible laptop pc intel celeron processor up to 2.48ghz...,iview maximus ii 11.6 laptop touchscreen 2-in-1 windows 10 intel bay trail z3735f processor 2gb ...


In [21]:
s1 = pd.merge(C1, C3, how='inner', on=['ltable_id','rtable_id'], suffixes=('',''))
len(s1)

ValueError: columns overlap but no suffix specified: Index(['_id', 'ltable_combo', 'rtable_combo'], dtype='object')

In [20]:
s1.head()

NameError: name 's1' is not defined

In [22]:
# Combine blocker outputs
C = em.combine_blocker_outputs_via_union([C2,C4])

In [23]:
len(C)

129324

We observe that the number of tuple pairs considered for matching is increased to 12530 (from 10165). Now let us debug the blocker output again to check if the current blocker sequence is dropping any potential matches.

In [ ]:
# Debug again
dbg = em.debug_blocker(C, A, B, output_size=10)

In [ ]:
dbg

We observe that the current blocker sequence does not drop obvious potential matches, and we can proceed with the matching step now. A subtle point to note here is, debugging blocker output practically provides a stopping criteria for modifying the blocker sequence.


# Matching tuple pairs in the candidate set

In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes.
This typically involves the following five steps:
1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set

## Sampling and labeling the candidate set

First, we randomly sample 450 tuple pairs for labeling purposes.

In [ ]:
# Sample  candidate set
S = em.sample_table(C, 48)
S.to_csv('s.csv')
S.head()

Next, we label the sampled candidate set. Specify we would enter 1 for a match and 0 for a non-match.

In [ ]:
# Label S
G = em.label_table(S, 'match')

For the purposes of this guide, we will load in a pre-labeled dataset (of 450 tuple pairs) included in this package.

In [ ]:
# # Load the pre-labeled data
# path_G = em.get_install_path() + os.sep + 'datasets' + os.sep + 'end-to-end' + os.sep + 'restaurants/lbl_restnt_wf1.csv'
# G = em.read_csv_metadata(path_G, 
#                          key='_id',
#                          ltable=A, rtable=B, 
#                          fk_ltable='ltable_id', fk_rtable='rtable_id')
# len(G)
G

## Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [ ]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

## Selecting the best learning-based matcher 

Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Converting the development set into feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation

### Creating a set of learning-based matchers

In [ ]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Creating features

Next, we need to create a set of features for the development set. *py_entitymatching* provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [ ]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=True)

In [ ]:
# List the names of the features generated
feature_table['feature_name']

### Converting the development set to feature vectors

In [ ]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='match',
                            show_progress=False)

In [ ]:
# Display first few rows
H.head(3)

### Selecting the best matcher using cross-validation

Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use 'precision' and 'recall' metric to select the best matcher.

In [ ]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
        k=5,
        target_attr='match', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

### Debugging matcher

We observe that the best matcher is not maximizing F1. We debug the matcher to see what might be wrong.
To do this, first we split the feature vectors into train and test.

In [ ]:
#  Split feature vectors into train and test
UV = em.split_train_test(H, train_proportion=0.5)
U = UV['train']
V = UV['test']

Next, we debug the matcher using GUI. For the purposes of this guide, we use random forest matcher for debugging purposes.

In [ ]:
# Debug decision tree using GUI
em.vis_debug_rf(rf, U, V, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
        target_attr='match')

From the GUI, we observe that phone numbers seem to be an important attribute, but they are in different format. Current features does not capture and adding a feature incorporating this difference in format can potentially improve 
the F1 numbers.

In [ ]:
def phone_phone_feature(ltuple, rtuple):
    p1 = ltuple.phone
    p2 = rtuple.phone
    p1 = p1.replace('/','-')
    p1 = p1.replace(' ','')
    p2 = p2.replace('/','-')
    p2 = p2.replace(' ','')    
    if p1 == p2:
        return 1.0
    else:
        return 0.0

In [ ]:
feature_table = em.get_features_for_matching(A, B)
em.add_blackbox_feature(feature_table, 'phone_phone_feature', phone_phone_feature)

Now, we repeat extracting feature vectors (this time with updated feature table), imputing table and selecting the best matcher again using cross-validation.

In [ ]:
H = em.extract_feature_vecs(I, feature_table=feature_table, attrs_after='gold', show_progress=False)

In [ ]:
# Select the best ML matcher using CV
result = em.select_matcher([dt, rf, svm, ln, lg, nb], table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'],
        k=5,
        target_attr='gold', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

Now, observe the best matcher is achieving a better F1. Let us stop here and  proceed on to evaluating the best matcher on the unseen data (the evaluation set).

##  Evaluating the matching output

Evaluating the matching outputs for the evaluation set typically involves the following four steps:
1. Converting the evaluation set to feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

### Converting the evaluation set to  feature vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set)

In [ ]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(J, feature_table=feature_table,
                            attrs_after='gold', show_progress=False)

### Training the selected matcher

Now, we train the matcher using all of the feature vectors from the development set. For the purposes of this guide we use random forest as the selected matcher.

In [ ]:
# Train using feature vectors from I 
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
       target_attr='gold')

### Predicting the matches

Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [ ]:
# Predict on L 
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'gold'], 
              append=True, target_attr='predicted', inplace=False)

### Evaluating the predictions

Finally, we evaluate the accuracy of predicted outputs

In [ ]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'gold', 'predicted')
em.print_eval_summary(eval_result)